In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data (assuming 'data' is a pandas DataFrame)
X = data[["x_turb2", "y_turb2", "wind_speed", "wind_direction", "turbulence_intensity"]].values
y = data["farm_power"].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
dataset = TensorDataset(X_train_tensor, y_train_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define Neural Network
class WindFarmNN(nn.Module):
    def __init__(self):
        super(WindFarmNN, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = WindFarmNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Evaluate model
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print(f"Test Loss: {test_loss.item():.4f}")

    # Find the input with the maximum predicted farm power
    max_index = torch.argmax(test_predictions)
    max_input = scaler.inverse_transform(X_test)[max_index]  # Scale back
    max_farm_power = test_predictions[max_index].item()
    print(f"Max Farm Power: {max_farm_power:.4f}")
    print(f"Corresponding Inputs: {max_input}")

    # Find the minimum (x_turb2, y_turb2) for given conditions
    X_test_original = scaler.inverse_transform(X_test)  # Scale back
    condition_mask = (X_test_original[:, 2] == 1) & (X_test_original[:, 3] == 270) & (X_test_original[:, 4] == 2)
    filtered_data = X_test_original[condition_mask]
    if filtered_data.shape[0] > 0:
        min_index = torch.argmin(test_predictions[condition_mask])
        min_x_turb2, min_y_turb2 = filtered_data[min_index, 0], filtered_data[min_index, 1]
        print(f"Minimum x_turb2: {min_x_turb2}, Minimum y_turb2: {min_y_turb2}")
    else:
        print("No matching data found for given conditions.")



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data (assuming 'data' is a pandas DataFrame)
X = data[["x_turb2", "y_turb2", "wind_speed", "wind_direction", "turbulence_intensity"]].values
y = data["farm_power"].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Get original min and max values for the parameters
X_min = X.min(axis=0)
X_max = X.max(axis=0)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, requires_grad=True)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32, requires_grad=True)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define Neural Network
class WindFarmNN(nn.Module):
    def __init__(self):
        super(WindFarmNN, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = WindFarmNN()
optimizer = optim.SGD([X_test_tensor], lr=0.01)  # We optimize the input features directly

# Run gradient descent to find maximum predicted farm power
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    predictions = model(X_test_tensor)
    negative_pred = -predictions  # Invert the predictions to maximize
    loss = negative_pred.mean()  # Minimize the negative prediction (maximize the original prediction)
    loss.backward()
    optimizer.step()
    
    # Convert X_min and X_max to PyTorch tensors and clamp the values
    min_tensor = torch.tensor(scaler.transform([X_min]).flatten(), dtype=torch.float32)
    max_tensor = torch.tensor(scaler.transform([X_max]).flatten(), dtype=torch.float32)
    
    # Clip the values to be within the original range to prevent extrapolation
    X_test_tensor.data = torch.clamp(X_test_tensor.data, min=min_tensor, max=max_tensor)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# After training, print the input with the maximum predicted farm power
model.eval()
with torch.no_grad():
    # Get the predictions for the optimized inputs
    optimized_predictions = model(X_test_tensor)
    
    # Find the index of the maximum predicted farm power
    max_farm_power, max_index = torch.max(optimized_predictions, dim=0)
    
    # Get the corresponding input that produced the max prediction
    max_input = X_test_tensor[max_index].detach().numpy()

    # Ensure max_input is 2D before applying inverse_transform
    max_input_reshaped = max_input.reshape(1, -1)  # Reshape to 2D array (1, 5)

    # Scale back the optimized input values to the original scale
    max_input_original_scale = scaler.inverse_transform(max_input_reshaped)

    print(f"Max Farm Power: {max_farm_power.item():.4f}")
    print(f"Corresponding Inputs (scaled back): {[f'{x:.6f}' for x in max_input_original_scale[0]]}")
